In [2]:
import numpy as np 
import roboticstoolbox as rtb 
from spatialmath import * 
from spatialmath.base import *
import spatialmath.base.symbolic as sym


In [10]:
alpha = sym.symbol("alpha")
beta = sym.symbol("beta")
A = sym.symbol("A")
mu = sym.symbol("mu")
km = sym.symbol("Km")

R1 = roty(alpha)
R2 = rotx(beta)
R_downz = rotx(np.pi)
R_prop = R1 @ R2 
T = np.array([0,0,-mu])
force_body = R_prop.dot(T)

print(force_body)

[-mu*sin(alpha)*cos(beta) mu*sin(beta) -mu*cos(alpha)*cos(beta)]


In [9]:
drag_torques = np.array([[0, 0 , 0 , 0],[0,0,0,0],[-km, -km, km , km]])
drag_torques = (R_downz.dot(drag_torques))
roll_arms = np.cross(np.array([[0, 0, 0, 0], [0,0,0,0], [-1,-1,-1,-1]]).T, np.array([[0, 0, 0, 0],[1,-1,-1,1],[0,0,0,0]]).T)
pitch_arms = np.cross(np.array([[0,0,0,0],[0, 0, 0, 0],  [-1,-1,-1,-1]]).T, np.array([[1, -1, 1, -1],[0,0,0,0],[0,0,0,0]]).T)
yaw_x_arms = np.cross(np.array([[0, 0, 0, 0],[1, 1, 1, 1],  [0,0,0,0]]).T , np.array([[1, -1, 1, -1], [0, 0, 0, 0 ], [0, 0, 0, 0]]).T)
yaw_y_arms = np.cross(np.array([[-1, -1, -1, -1],[0, 0, 0, 0],[0,0,0,0]]).T, np.array([[0, 0, 0, 0],[1, -1,-1, 1], [0,0,0,0]]).T)
#print(roll_arms[:,0].T * A , pitch_arms[:, 1].T * A, yaw_x_arms)
roll_torques = force_body.dot(np.array([drag_torques[2], [0,0,0,0],roll_arms[:,0].T * A]))
pitch_torques = force_body.dot(np.array([[0,0, 0 , 0], drag_torques[2],pitch_arms[:, 1].T * A]))
yaw_torques = force_body.dot(np.array([yaw_y_arms[:, 2].T * A,yaw_x_arms[:, 2].T * A,[0, 0 , 0 , 0]]) + drag_torques)
print(roll_torques)
print(" ")
print(pitch_torques)
print("")
print(yaw_torques)

[-A*cos(alpha)*cos(beta) - 1.0*Km*sin(alpha)*cos(beta)
 A*cos(alpha)*cos(beta) - 1.0*Km*sin(alpha)*cos(beta)
 A*cos(alpha)*cos(beta) + 1.0*Km*sin(alpha)*cos(beta)
 -A*cos(alpha)*cos(beta) + 1.0*Km*sin(alpha)*cos(beta)]
 
[A*cos(alpha)*cos(beta) + 1.0*Km*sin(beta)
 -A*cos(alpha)*cos(beta) + 1.0*Km*sin(beta)
 A*cos(alpha)*cos(beta) - 1.0*Km*sin(beta)
 -A*cos(alpha)*cos(beta) - 1.0*Km*sin(beta)]

[A*sin(alpha)*cos(beta) - 1.0*Km*cos(alpha)*cos(beta) + (-A + 1.22464679914735e-16*Km)*sin(beta)
 -A*sin(alpha)*cos(beta) - 1.0*Km*cos(alpha)*cos(beta) + (A + 1.22464679914735e-16*Km)*sin(beta)
 -A*sin(alpha)*cos(beta) + 1.0*Km*cos(alpha)*cos(beta) + (-A - 1.22464679914735e-16*Km)*sin(beta)
 A*sin(alpha)*cos(beta) + 1.0*Km*cos(alpha)*cos(beta) + (A - 1.22464679914735e-16*Km)*sin(beta)]


In [25]:
forces_body = np.tile(force_body, (4, 1))
torques = np.cross(forces_body, np.array([[0.13, 0.13,0],[-0.13, -0.13, 0], [ 0.13, -0.13, 0], [-0.13, 0.13, 0]])).T
print(torques)

[[0.13*mu*cos(alpha)*cos(beta) -0.13*mu*cos(alpha)*cos(beta)
  -0.13*mu*cos(alpha)*cos(beta) 0.13*mu*cos(alpha)*cos(beta)]
 [-0.13*mu*cos(alpha)*cos(beta) 0.13*mu*cos(alpha)*cos(beta)
  -0.13*mu*cos(alpha)*cos(beta) 0.13*mu*cos(alpha)*cos(beta)]
 [-0.13*mu*sin(alpha)*cos(beta) - 0.13*mu*sin(beta)
  0.13*mu*sin(alpha)*cos(beta) + 0.13*mu*sin(beta)
  0.13*mu*sin(alpha)*cos(beta) - 0.13*mu*sin(beta)
  -0.13*mu*sin(alpha)*cos(beta) + 0.13*mu*sin(beta)]]


In [28]:
drag_torques[1] = 0
#print(drag_torques)
drag_torques_body = R_prop.dot(drag_torques)
print(torques + drag_torques_body)

[[1.0*Km*sin(alpha)*cos(beta) + 0.13*mu*cos(alpha)*cos(beta)
  1.0*Km*sin(alpha)*cos(beta) - 0.13*mu*cos(alpha)*cos(beta)
  -1.0*Km*sin(alpha)*cos(beta) - 0.13*mu*cos(alpha)*cos(beta)
  -1.0*Km*sin(alpha)*cos(beta) + 0.13*mu*cos(alpha)*cos(beta)]
 [-1.0*Km*sin(beta) - 0.13*mu*cos(alpha)*cos(beta)
  -1.0*Km*sin(beta) + 0.13*mu*cos(alpha)*cos(beta)
  1.0*Km*sin(beta) - 0.13*mu*cos(alpha)*cos(beta)
  1.0*Km*sin(beta) + 0.13*mu*cos(alpha)*cos(beta)]
 [1.0*Km*cos(alpha)*cos(beta) - 0.13*mu*sin(alpha)*cos(beta) - 0.13*mu*sin(beta)
  1.0*Km*cos(alpha)*cos(beta) + 0.13*mu*sin(alpha)*cos(beta) + 0.13*mu*sin(beta)
  -1.0*Km*cos(alpha)*cos(beta) + 0.13*mu*sin(alpha)*cos(beta) - 0.13*mu*sin(beta)
  -1.0*Km*cos(alpha)*cos(beta) - 0.13*mu*sin(alpha)*cos(beta) + 0.13*mu*sin(beta)]]
